In [10]:
import pandas as pd
from glob import glob
import os
from os.path import join
import numpy as np

In [36]:
dataPath = "rawData"
files = glob(join(dataPath, "snb-data*"))
# files.remove("rawData/snb-data-foreignCurrencyInvestment.csv")
files

['rawData/snb-data-reservePositions.csv',
 'rawData/snb-data-foreignCurrencyInvestment.csv',
 'rawData/snb-data-monetaryAssistanceLoans.csv',
 'rawData/snb-data-CPI.csv',
 'rawData/snb-data-exchangeRate.csv',
 'rawData/snb-data-callMoneyRate.csv']

In [53]:
df = []

for f in files:
    dfFile = pd.read_csv(f, skiprows = 3, sep = ";", index_col = 0)
    if "D1" in dfFile.columns:
        dfFile["D0"] = dfFile["D0"] + "_" + dfFile["D1"]
    dfFile = dfFile.pivot(columns = "D0", values = "Value")
 
    if f == "rawData/snb-data-foreignCurrencyInvestment.csv":
        dfFile.index = pd.to_datetime(dfFile.index).strftime("%Y-%m")

    df.append(dfFile)
df = pd.concat(df, axis = 1).sort_index()
df

D0,RAFAFX,RAFAGOLD,RAFAIMF,RAFAO,RAFASDR,RAXG,ICHF0,ICHF1,ICHF2,ICHF3,...,3M2,3M3,3M4,EG3M,ESTR,EURIBOR,SARON,SOFR,SONIA,TONA
Date,,,,,,,,,,,,,,,,,,,,,
1914-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03,715624.526372,66756.193747,1913.979440,-425.091831,11073.517905,728186.931886,NaN,NaN,NaN,NaN,...,NaN,5.3037,NaN,1.589,3.899,3.892,1.464207,5.34,5.1911,0.074
2024-04,720306.831437,70640.862862,1863.213446,-10.945579,11077.121241,733236.220546,280265.205531,273363.536399,59045.650642,45520.089773,...,NaN,NaN,NaN,1.307,3.889,3.825,1.444518,5.34,5.1998,0.076
2024-05,717694.593856,70773.436203,1804.181481,0.252651,11096.026584,730595.054573,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.298,3.892,3.785,1.450122,5.34,5.2000,0.077


In [76]:
# Y
eurchf_fx_df = df[["M0_EUR1"]]

# X1
chf_callmoneyrate_df = df[["SARON"]]

# X2
x2_total_names = ["T0", "T1", "T2", "T3", "T4"]
x2_chf_names = ["ICHF0", "ICHF1", "ICHF2", "ICHF3", "ICHF4", "ICHF5", "ICHF6"]
total_currency_investment_df = df[x2_total_names].ffill().sum(axis=1)
chf_foreign_currency_investment_df = df[x2_chf_names].ffill().sum(axis=1)
foreign_currency_investment_df = total_currency_investment_df - chf_foreign_currency_investment_df
proportion_foreign_currency_investment_df = foreign_currency_investment_df / total_currency_investment_df

# X3
chf_reserve_positions_imf_df = df[["RIWF"]].ffill().sum(axis=1)
chf_total_assets_df = df[["T0"]].ffill().sum(axis=1)
chf_proportion_reserve_positions_imf_df = chf_reserve_positions_imf_df / chf_total_assets_df

# X4
chf_monetary_assitance_loans_df = df[["W"]].ffill().sum(axis=1)
chf_total_assets_df = df[["T0"]].ffill().sum(axis=1)
chf_proportion_monetary_assitance_loans_df = chf_monetary_assitance_loans_df / chf_total_assets_df

# X5
chf_secuties_df = df[["WSF"]].ffill().sum(axis=1)
chf_total_assets_df = df[["T0"]].ffill().sum(axis=1)
chf_proportion_secuties_df = chf_secuties_df / chf_total_assets_df

# X6
chf_other_assets_df = df[["UA"]].ffill().sum(axis=1)
chf_total_assets_df = df[["T0"]].ffill().sum(axis=1)
chf_proportion_other_assets_df = chf_other_assets_df / chf_total_assets_df

# X7
chf_gdp_q_sa_index_df = pd.read_csv(join(dataPath, "CPMNACSAB1GQCH.csv"), index_col = 0)

# X8
eur_gdp_q_sa_index_df = pd.read_csv(join(dataPath, "EUNNGDP.csv"), index_col = 0)

# X9
chf_cpi_yoy_df = df[["TLK"]]